In [1]:
import pandas as pd



In [8]:
df = pd.read_csv('./employees.csv', parse_dates=['hire_date'])
df

,employee_id,department,manager,hire_date,salary,region,sales
0,101,Sales,Alice,2018-05-10,70000,East,150000
1,102,Sales,Alice,2020-07-12,68000,East,120000
2,103,Engineering,Bob,2019-03-05,95000,West,50000
3,104,Engineering,Bob,2021-11-20,87000,West,60000
4,105,HR,Carol,2017-01-15,60000,East,0
5,106,HR,Carol,2019-08-22,62000,West,0
6,107,Sales,Alice,2022-04-01,71000,West,80000
7,108,Engineering,Bob,2018-09-09,98000,East,70000
8,109,Sales,Alice,2019-12-17,69000,West,90000
9,110,HR,Carol,2021-06-05,64000,East,0


# Q1. count employees by department

"I’ll start by grouping by the department column and counting employees. In pandas, .size() counts rows, which is appropriate here. Then I’ll reset the index for readability."

In [ ]:

df.groupby('department')['employee_id'].count().reset_index(name='num_employees')

,department,num_employees
0,Engineering,3
1,HR,3
2,Sales,4


In [13]:
df.groupby('department').size().reset_index(name='num_employees')

,department,num_employees
0,Engineering,3
1,HR,3
2,Sales,4


# Q2. average salary per department

Next, I’ll compute the average salary per department. This is a straightforward aggregation with .mean()."

"I prefer resetting the index so the result is a clean DataFrame, not a Series with department as the index."

In [ ]:

df.groupby('department')['salary'].mean().reset_index(name='avg_salary')

,department,avg_salary
0,Engineering,93333.333333
1,HR,62000.000000
2,Sales,69500.000000


,employee_id,department,manager,hire_date,salary,region,sales
0,101,Sales,Alice,2018-05-10,70000,East,150000
1,102,Sales,Alice,2020-07-12,68000,East,120000
2,103,Engineering,Bob,2019-03-05,95000,West,50000
3,104,Engineering,Bob,2021-11-20,87000,West,60000
4,105,HR,Carol,2017-01-15,60000,East,0
5,106,HR,Carol,2019-08-22,62000,West,0
6,107,Sales,Alice,2022-04-01,71000,West,80000
7,108,Engineering,Bob,2018-09-09,98000,East,70000
9,110,HR,Carol,2021-06-05,64000,East,0


# Q3. total and average sales per region 

"Here I need multiple aggregations: total and average sales. With .agg(), I can specify both functions and rename columns using keyword arguments."

In [18]:
df.groupby('region')['sales'].agg(
  total_sales='sum',
  avg_sales='mean'
).reset_index()

,region,total_sales,avg_sales
0,East,340000,68000.0
1,West,280000,56000.0


In [ ]:

df.groupby('region').agg(
  total_sales=('sales', 'sum'),
  avg_sales=('sales', 'mean')
).reset_index()

,region,total_sales,avg_sales
0,East,340000,68000.0
1,West,280000,56000.0


# Q4. earliest and latest hire date per manager

Since hire_date is parsed as a datetime, I can take min and max. I’ll group by manager, then aggregate both."

In [ ]:

df.groupby('manager')['hire_date'].agg(
  earliest_hire='min',
  latest_hire='max'
).reset_index()

,manager,earliest_hire,latest_hire
0,Alice,2018-05-10,2022-04-01
1,Bob,2018-09-09,2021-11-20
2,Carol,2017-01-15,2021-06-05


# Q5. distinct count of employees per department and region

This is a multi-key groupby, and I want the number of unique employees. That’s a good use case for .nunique()."

In [ ]:

df.groupby(['department', 'region'])['employee_id'].nunique().reset_index(name='distinct_employee_count')

,department,region,distinct_employee_count
0,Engineering,East,1
1,Engineering,West,2
2,HR,East,2
3,HR,West,1
4,Sales,East,2
5,Sales,West,2


# Q6. for each department, list the top 1 highest paid employees

To find the highest paid per department, I’ll sort by salary descending, then take the first row in each group using groupby().head(1)."

This way, for each department, I get the employee(s) with the max salary. If there are ties, it’ll include them too."

In [ ]:

df.sort_values('salary', ascending=False).groupby('department').head(1)


,employee_id,department,manager,hire_date,salary,region,sales
7,108,Engineering,Bob,2018-09-09,98000,East,70000
6,107,Sales,Alice,2022-04-01,71000,West,80000
9,110,HR,Carol,2021-06-05,64000,East,0


# Q7. For each manager, collect employee IDs as a comma-separated string

Now I’ll group by manager, and aggregate employee IDs. To combine them into a string, I’ll use apply with join."

In [ ]:

df.groupby('manager')['employee_id'].apply(
  lambda x: ','.join(map(str, sorted(x)))
).reset_index(name='employee_list')

,manager,employee_list
0,Alice,"101,102,107,109"
1,Bob,"103,104,108"
2,Carol,"105,106,110"


# Q8. For each region, what % of total company sales does it contribute?

Finally, I’ll compute sales by region, then divide by the total company sales to get percentages."

This gives business insight into how much each region contributes relative to the whole."

In [ ]:

region_sales = df.groupby('region')['sales'].sum().reset_index()
region_sales['pct_of_total'] = region_sales['sales']/region_sales['sales'].sum()*100
region_sales

,region,sales,pct_of_total
0,East,340000,54.83871
1,West,280000,45.16129


These are the key aggregations and insights I’d extract from the dataset. I focused on clean, readable groupby operations and made sure results are returned as DataFrames for clarity. Let me know if you’d like me to optimize further or format outputs differently."

#### curve ball stretch questions

# Q9. Monthly average salary trend per department

shows you know how to use pd.Grouper for time-based aggregation.

In [42]:
df

,employee_id,department,manager,hire_date,salary,region,sales
0,101,Sales,Alice,2018-05-10,70000,East,150000
1,102,Sales,Alice,2020-07-12,68000,East,120000
2,103,Engineering,Bob,2019-03-05,95000,West,50000
3,104,Engineering,Bob,2021-11-20,87000,West,60000
4,105,HR,Carol,2017-01-15,60000,East,0
5,106,HR,Carol,2019-08-22,62000,West,0
6,107,Sales,Alice,2022-04-01,71000,West,80000
7,108,Engineering,Bob,2018-09-09,98000,East,70000
8,109,Sales,Alice,2019-12-17,69000,West,90000
9,110,HR,Carol,2021-06-05,64000,East,0


Let’s resample salaries by department on a monthly basis. That means grouping by both department and hire month."

This shows average salary trends per department by month of hire."

In [44]:
df.groupby([
  'department',
  pd.Grouper(key='hire_date', freq='ME')
])['salary'].mean().reset_index()

,department,hire_date,salary
0,Engineering,2018-09-30,98000.0
1,Engineering,2019-03-31,95000.0
2,Engineering,2021-11-30,87000.0
3,HR,2017-01-31,60000.0
4,HR,2019-08-31,62000.0
5,HR,2021-06-30,64000.0
6,Sales,2018-05-31,70000.0
7,Sales,2019-12-31,69000.0
8,Sales,2020-07-31,68000.0
9,Sales,2022-04-30,71000.0


# Q10. Pivot table of total sales by department and region

demonstrates pivot table fluency (common business request).

In [45]:
pd.pivot_table(
  df, 
  index='department',
  columns='region',
  values='sales',
  aggfunc='sum',
  fill_value=0
)

region,East,West
department,,
Engineering,70000,110000
HR,0,0
Sales,270000,170000


# Q11. Top 2 employees by sales in each region

In [46]:
df.sort_values('sales', ascending=False).groupby('region').head(2)

,employee_id,department,manager,hire_date,salary,region,sales
0,101,Sales,Alice,2018-05-10,70000,East,150000
1,102,Sales,Alice,2020-07-12,68000,East,120000
8,109,Sales,Alice,2019-12-17,69000,West,90000
6,107,Sales,Alice,2022-04-01,71000,West,80000


# Q12. rolling average of sales per region (last 2 hires)

In [58]:
df_sorted = df.sort_values(['region', 'hire_date'])
df_sorted

,employee_id,department,manager,hire_date,salary,region,sales
4,105,HR,Carol,2017-01-15,60000,East,0
0,101,Sales,Alice,2018-05-10,70000,East,150000
7,108,Engineering,Bob,2018-09-09,98000,East,70000
1,102,Sales,Alice,2020-07-12,68000,East,120000
9,110,HR,Carol,2021-06-05,64000,East,0
2,103,Engineering,Bob,2019-03-05,95000,West,50000
5,106,HR,Carol,2019-08-22,62000,West,0
8,109,Sales,Alice,2019-12-17,69000,West,90000
3,104,Engineering,Bob,2021-11-20,87000,West,60000
6,107,Sales,Alice,2022-04-01,71000,West,80000


In [63]:
df_sorted.groupby('region')['sales'].rolling(2).mean().reset_index()

,region,level_1,sales
0,East,4,NaN
1,East,0,75000.0
2,East,7,110000.0
3,East,1,95000.0
4,East,9,60000.0
5,West,2,NaN
6,West,5,25000.0
7,West,8,45000.0
8,West,3,75000.0
9,West,6,70000.0


In [62]:
df_sorted.groupby('region')['sales'].rolling(2, min_periods=1).mean().reset_index()

,region,level_1,sales
0,East,4,0.0
1,East,0,75000.0
2,East,7,110000.0
3,East,1,95000.0
4,East,9,60000.0
5,West,2,50000.0
6,West,5,25000.0
7,West,8,45000.0
8,West,3,75000.0
9,West,6,70000.0


# Q13. Percentage of employees in each department relative to company

In [70]:
emp_cnt_dept = df.groupby('department')['employee_id'].nunique().reset_index(name='employee_cnt')
emp_cnt_dept

,department,employee_cnt
0,Engineering,3
1,HR,3
2,Sales,4


In [71]:
emp_cnt_dept['pct'] = emp_cnt_dept['employee_cnt']/emp_cnt_dept['employee_cnt'].sum()*100
emp_cnt_dept

,department,employee_cnt,pct
0,Engineering,3,30.0
1,HR,3,30.0
2,Sales,4,40.0


In [72]:
df["department"].value_counts().reset_index()

,department,count
0,Sales,4
1,Engineering,3
2,HR,3
